In [5]:
import pandas as pd 

In [6]:
movies_df = pd.read_csv("ml-latest-small/movies.csv")
ratings_df = pd.read_csv("ml-latest-small/ratings.csv")

In [7]:
n_users = len(ratings_df.userId.unique())
n_items = len(ratings_df.movieId.unique())

In [4]:
import torch
import torch.nn as nn

class UserTower(nn.Module):
    def __init__(self, num_users, embedding_dim):
        super(UserTower, self).__init__()
        self.embedding = nn.Embedding(num_users, embedding_dim)
        self.fc1 = nn.Linear(embedding_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.relu = nn.ReLU()

    def forward(self, user_ids):
        user_embeddings = self.embedding(user_ids)
        x = self.relu(self.fc1(user_embeddings))
        user_features = self.fc2(x)
        return user_features

class MovieTower(nn.Module):
    def __init__(self, num_movies, embedding_dim):
        super(MovieTower, self).__init__()
        self.embedding = nn.Embedding(num_movies, embedding_dim)
        self.fc1 = nn.Linear(embedding_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.relu = nn.ReLU()

    def forward(self, movie_ids):
        movie_embeddings = self.embedding(movie_ids)
        x = self.relu(self.fc1(movie_embeddings))
        movie_features = self.fc2(x)
        return movie_features

class TwoTowerRecommender(nn.Module):
    def __init__(self, num_users, num_movies, embedding_dim):
        super(TwoTowerRecommender, self).__init__()
        self.user_tower = UserTower(num_users, embedding_dim)
        self.movie_tower = MovieTower(num_movies, embedding_dim)
        self.rating_layer = nn.Linear(32 + 32, 1)

    def forward(self, user_ids, movie_ids):
        user_features = self.user_tower(user_ids)
        movie_features = self.movie_tower(movie_ids)
        combined_features = torch.cat([user_features, movie_features], dim=1)
        ratings = self.rating_layer(combined_features)
        return ratings.squeeze()

In [ ]:
# Instantiate the model
model = TwoTowerRecommender(num_users=n_users, num_movies=n_items, embedding_dim=32)

# Define the loss function and optimizer
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

num_epochs = 128
# Training loop
for epoch in range(num_epochs):
    optimizer.zero_grad()
    predicted_ratings = model(n_users, n_items)
    loss = loss_fn(predicted_ratings, ratings)
    loss.backward()
    optimizer.step()
    print(f"Epoch: {epoch}, Loss: {loss.item()}")